In [1]:
# connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Define directories
save_dir = "/content/drive/MyDrive/2025 Fall/PhysicsExam/"

In [3]:
# Imports + Initialization
import matplotlib
matplotlib.use("Agg")

import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio

In [4]:
# Numerical Parameters
hbar = 1.0
m = 1.0
z_min, z_max = -400, 400
N = 4096
z = np.linspace(z_min, z_max, N)
dz = z[1] - z[0]

k = 2*np.pi * np.fft.fftfreq(N, d=dz)

dt = 0.005
steps = 40000  # Increased to ensure packets reach z=200

In [5]:
# Initial wave packet
z0 = -100        # starting position
sigma = 2
k0 = 3.0

psi0 = np.exp(-(z - z0)**2/(2*sigma**2)) * np.exp(1j*k0*z)
psi0 /= np.sqrt(np.sum(np.abs(psi0)**2) * dz)

In [32]:
# Unperturbed Potential
V0 = np.zeros_like(z)

# Perturbed potential
A = 1.4
B = 2.05
sigma_bump = 2.0
sigma_well = 3.0

V = (
    A*np.exp(-(z-4.2)**2/sigma_bump**2) +
    A*np.exp(-(z+4.2)**2/sigma_bump**2) -
    B*np.exp(-z**2/sigma_well**2)
)

# # For fun test
# V = (
#     2*np.exp(-(z-4.2)**2/3**2) +
#     2*np.exp(-(z+4.2)**2/3**2)
# )

In [33]:
# Split Operator Phases

K_phase = np.exp(-1j * (hbar*k**2)/(2*m) * dt)

def potential_phase(V):
    return np.exp(-1j * V * dt / (2*hbar))

V_phase0 = potential_phase(V0)
V_phaseP = potential_phase(V)

In [34]:
# Evolution

def evolve(V_phase, target_z=200):
    psi = psi0.copy()

    psi_start = psi.copy()
    psi_mid = None
    psi_end = None
    arrival = None

    for step in range(steps):

        if step == steps//2:
            psi_mid = psi.copy()

        psi *= V_phase
        psi_k = np.fft.fft(psi)
        psi_k *= K_phase
        psi = np.fft.ifft(psi_k)
        psi *= V_phase
        psi /= np.sqrt(np.sum(np.abs(psi)**2) * dz)

        z_mean = np.sum(z * np.abs(psi)**2) * dz

        if arrival is None and z_mean >= target_z:
            arrival = step * dt
            psi_end = psi.copy()
            break

    if psi_end is None:
        psi_end = psi.copy()

    return arrival, psi_start, psi_mid, psi_end

In [35]:
# Run Simulation

T0, psi0_start, psi0_mid, psi0_end = evolve(V_phase0, target_z=200)
T , psiP_start, psiP_mid, psiP_end = evolve(V_phaseP, target_z=200)

print("Unperturbed arrival time T0:", T0 if T0 is not None else "Did not reach z=200")
print("Perturbed arrival time T :", T if T is not None else "Did not reach z=200")
if T0 is not None and T is not None:
    print("Delay ΔT =", T - T0)
else:
    print("Warning: One or both packets did not reach z=200 in time!")

Unperturbed arrival time T0: 100.0
Perturbed arrival time T : 101.455
Delay ΔT = 1.4549999999999983


In [36]:
# Plot Simulation

def plot_snapshots(z, s, m, e, title):
    plt.figure(figsize=(12,6))
    plt.plot(z, np.abs(s)**2, label="Start")
    plt.plot(z, np.abs(m)**2, label="Midway")
    plt.plot(z, np.abs(e)**2, label="End")
    plt.xlabel("z")
    plt.ylabel("|ψ(z)|²")
    plt.title(title)
    plt.grid(True)
    plt.legend()
    plt.show()

plot_snapshots(z, psi0_start, psi0_mid, psi0_end, "Unperturbed Wave Packet")
plot_snapshots(z, psiP_start, psiP_mid, psiP_end, "Perturbed Wave Packet")

In [37]:
# TDSE EVOLUTION FOR GIF (STOPS AT z_mean >= 200)

def evolve_frames(V_phase, frame_skip=50):
    psi = psi0.copy()
    frames = []
    times = []
    z_means = []

    step = 0
    while step < steps:

        # Compute current center position
        z_mean = np.sum(z * np.abs(psi)**2) * dz

        # Check if we've reached z=200 BEFORE doing anything else
        if z_mean >= 200:
            # Save final frame and stop
            frames.append(psi.copy())
            times.append(step * dt)
            z_means.append(z_mean)
            break

        # Save frame at regular intervals
        if step % frame_skip == 0:
            frames.append(psi.copy())
            times.append(step * dt)
            z_means.append(z_mean)

        # TDSE evolution step
        psi *= V_phase
        psi_k = np.fft.fft(psi)
        psi_k *= K_phase
        psi = np.fft.ifft(psi_k)
        psi *= V_phase
        psi /= np.sqrt(np.sum(np.abs(psi)**2) * dz)

        step += 1

    return frames, times, z_means


# Run evolutions
frames0, times0, _ = evolve_frames(V_phase0)
framesP, timesP, _ = evolve_frames(V_phaseP)

print(f"Unperturbed GIF ends at t = {times0[-1]:.3f}")
print(f"Perturbed GIF ends at t = {timesP[-1]:.3f}")

Unperturbed GIF ends at t = 100.005
Perturbed GIF ends at t = 101.460


In [38]:
# Save gifs and output results

def save_gif(z, psi_frames, times, filename, use_potential=True):
    images = []

    psi2_max = max(np.max(np.abs(f)**2) for f in psi_frames)
    Vmin, Vmax = np.min(V), np.max(V)

    for psi, t in zip(psi_frames, times):
        psi2 = np.abs(psi)**2
        psiRe = np.real(psi)

        fig, ax1 = plt.subplots(figsize=(10,4))
        ax1.set_xlim(-400, 400)

        # wave
        ax1.plot(z, psi2, color='blue', label='|ψ|²')
        if np.max(np.abs(psiRe)) > 0:
            scale = psi2_max * 0.7 / np.max(np.abs(psiRe))
            ax1.plot(z, psiRe * scale, color='black', alpha=0.6,
                     label='Re(ψ) (scaled)')

        ax1.set_xlabel("z")
        ax1.set_ylabel("|ψ(z)|², Re(ψ)", color='blue')
        ax1.set_ylim(0, psi2_max*1.2)
        ax1.tick_params(axis='y', labelcolor='blue')

        # potential
        ax2 = ax1.twinx()
        if use_potential:
            ax2.plot(z, V, color='red', alpha=0.7, label='V(z)')
            ax2.set_ylim(Vmin*1.2, Vmax*1.2)
        else:
            ax2.plot(z, 0*z, color='red', alpha=0.7, label='V(z)=0')
            ax2.set_ylim(-1, 1)
        ax2.set_ylabel("V(z)", color='red')
        ax2.tick_params(axis='y', labelcolor='red')

        # TIME LABEL
        ax1.text(0.02, 0.92, f"t = {t:.3f}",
                 transform=ax1.transAxes,
                 fontsize=14, color='black',
                 bbox=dict(facecolor='white', alpha=0.7))

        # legend
        L1, L2 = ax1.get_legend_handles_labels()
        L3, L4 = ax2.get_legend_handles_labels()
        ax1.legend(L1+L3, L2+L4, loc="upper right")

        plt.title(filename.replace(".gif",""))
        fig.tight_layout()
        fig.canvas.draw()

        images.append(np.asarray(fig.canvas.buffer_rgba()))
        plt.close(fig)

    imageio.mimsave(save_dir + filename, images, fps=30)

# unperturbed GIF: flat potential
save_gif(z, frames0, times0, "unperturbed.gif", use_potential=False)

# perturbed GIF: real potential
save_gif(z, framesP, timesP, "perturbed.gif", use_potential=True)

print("GIFs saved to:", save_dir)

GIFs saved to: /content/drive/MyDrive/2025 Fall/PhysicsExam/


In [31]:
# ============================================================
# COMPUTE & SAVE A(k) PLOT
# This section (everything below) was made by ChatGPT
# to make sure my solution is the same as the analytical
# ============================================================

# 1. Numerical A(k)
A_k_num = np.fft.fftshift(np.fft.fft(psi0))
dk = k[1] - k[0]

# Normalize
A_k_num /= np.sqrt(np.sum(np.abs(A_k_num)**2) * abs(dk))

# 2. Shift the k array
k_shift = np.fft.fftshift(k)

# 3. Analytic A(k)
A_k_analytic = (
    sigma * np.sqrt(2*np.pi) *
    np.exp(-0.5 * sigma**2 * (k_shift - k0)**2) *
    np.exp(-1j * (k_shift - k0) * z0)
)

# Normalize analytic
A_k_analytic /= np.sqrt(np.sum(np.abs(A_k_analytic)**2) * abs(k_shift[1] - k_shift[0]))

# 4. PLOT (no xlim yet — full range)
plt.figure(figsize=(12,6))
plt.plot(k_shift, np.abs(A_k_num)**2, label="Numerical |A(k)|²", color="blue")
plt.plot(k_shift, np.abs(A_k_analytic)**2, label="Analytic |A(k)|²", color="red", linestyle="--")
plt.xlabel("k")
plt.ylabel("|A(k)|²")
plt.title("Momentum-Space Amplitude A(k)")
plt.grid(True)
plt.legend()

# 5. SAVE PLOT
plt.savefig(save_dir + "A_k_plot.png", dpi=300)
plt.close()

print("Saved plot to:", save_dir + "A_k_plot.png")


Saved plot to: /content/drive/MyDrive/2025 Fall/PhysicsExam/A_k_plot.png
